In [37]:
import os
import glob

import cv2
import itk
import numpy as np

In [38]:
def convert(fname, thresh):
    imlist = []

    cap = cv2.VideoCapture(fname)
    if cap.isOpened():

        image_type = itk.Image[itk.UC,2]
        diff_filter = itk.SimilarityIndexImageFilter[image_type,image_type].New()

        count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            img_slice = itk.image_from_array(np.array(gray))
            if count == 0:
                base_img_slice = img_slice
                imlist.append(img_slice)
                count += 1
            else:
                diff_filter.SetInput1(base_img_slice)
                diff_filter.SetInput2(img_slice)
                diff_filter.Update()
                indx = diff_filter.GetSimilarityIndex()
                if indx < thresh:
                    base_img_slice = img_slice
                    imlist.append(img_slice)
                    count += 1
        cap.release()
 
    return imlist

In [45]:
def create_sparse_image(fname, target_count):
    count = 0
    thresh = 1.0
    low_thresh = 1.0
    step = 0.1
    step_dir = -1
    while count != target_count and step > 1e-10 and thresh > 0.5:
        imlist = convert(fname,thresh)
        count = len(imlist)
        if step_dir == 1 and count > target_count:
            step_dir = -1
            step = step * 0.5
        elif step_dir == -1 and count < target_count:
                low_thresh = thresh
                step_dir = 1
                step = step * 0.5
        thresh += step_dir * step
    if count > 10:
        print('   ERROR: Did not attain target count =', target_count)
        imlist = convert(fname,low_thresh)
        count = len(imlist)
        print('   Creating image with', count, 'slices using threshold =', low_thresh)
    else:
        print('   Success!')
    dir_name = 'MHA'
    if os.path.isdir(dir_name):
        fileList = glob.glob(dir_name+'/'+fname+'-?????.mha')
        for file in fileList:
            os.remove(file)
    else:
        os.mkdir(dir_name)
    for i in range(len(imlist)):
        outfname = fname+'-%05d.mha' % i
        itk.imwrite(imlist[i], dir_name+'/'+outfname)

In [46]:
create_sparse_image('Vir_liftl_H1N1_case2_clip2.mp4',10)

   ERROR: Did not attain target count = 10
   Creating image with 4 slices using threshold = 0.9999999996274712


In [47]:
for name in glob.glob('*.[gm][ip][f4]'):
    print(name)
    create_sparse_image(name,10)

Vir_prospective_file2.mp4
   Success!
Cov_emdocs_vid2.gif
   Success!
Reg-nephropocus.mp4
   Success!
Vir_prospective_file1.mp4
   Success!
Vir_liftl_H1N1_case2_clip2.mp4
   ERROR: Did not attain target count = 10
   Creating image with 4 slices using threshold = 0.9999999996274712
Reg_Image_18122_crop.mp4
   Success!
Reg_Image_181432_crop.mp4
   Success!
Reg_Image_181206_crop.mp4
   Success!
Reg-minimalmovement.mp4
   Success!
Reg-grep-normal-alines-original.mp4
   Success!
Reg-Youtube-start20sec.mp4
   Success!
Reg-NormalLung.mp4
   Success!
Reg-Youtube-Video_902_Lung_POCUS-left.mp4
   Success!
Pneu_prospective_file3.mp4
   ERROR: Did not attain target count = 10
   Creating image with 9 slices using threshold = 0.9998816408216952
Pneu-Youtube-start20sec.mp4
   Success!
Cov_linear_abrams_2020_v1.mp4
   Success!
Cov-Atlas-pleural2.mp4
   Success!
Cov-grep-7432.mp4
   Success!
Cov-grep-7500.mp4
   Success!
Cov-grep-7431.mp4
   Success!
Cov-Atlas-pleural.mp4
   Success!
